<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/EfficientNetV2L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
import os
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
from keras import metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
import random
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
img_size = 512

In [4]:
base_model = EfficientNetV2L(weights='imagenet', include_top=False,
                            input_shape=(600,600,3))
# freeze extraction layers
base_model.trainable = False

# add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(4096,activation="relu")(x)
x = Dense(4096,activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(2096,activation="relu")(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# confirm unfrozen layers
for layer in model.layers:
    if layer.trainable==True:
        print(layer)

473176280/473176280 [==============================] - 6s 0us/step


In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 600, 600, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 600, 600, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 300, 300, 32  864         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [6]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=1),
                ModelCheckpoint('model.hdf5',
                                 save_best_only=True)]

In [7]:
opt = Adam(learning_rate=0.005)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy', metrics.Precision(), metrics.Recall()]
)

In [8]:
image_generator = ImageDataGenerator(rescale = 1.0/255.0,
                                   rotation_range = 0.5,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
                                  )

image_generator_test = ImageDataGenerator(rescale = 1.0/255.0)


In [9]:
train_data = image_generator.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/train',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 1717 images belonging to 2 classes.


In [10]:
test_data = image_generator_test.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/test',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 431 images belonging to 2 classes.


In [11]:
history=model.fit(train_data,
                  epochs=50,
                  validation_data=test_data,
                  validation_steps=len(test_data),
                  verbose=1,
                  callbacks=callbacks)

Epoch 1/50
54/54 [==============================] - 514s 9s/step - loss: 6.7674 - accuracy: 0.5422 - precision: 0.5338 - recall: 0.5294 - val_loss: 0.6789 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 2/50
54/54 [==============================] - 178s 3s/step - loss: 0.7142 - accuracy: 0.5475 - precision: 0.5507 - recall: 0.5405 - val_loss: 0.6856 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 3/50
54/54 [==============================] - 175s 3s/step - loss: 0.8204 - accuracy: 0.5719 - precision: 0.5720 - recall: 0.5644 - val_loss: 0.6835 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 4/50
54/54 [==============================] - 175s 3s/step - loss: 0.7590 - accuracy: 0.5731 - precision: 0.5728 - recall: 0.5725 - val_loss: 0.6832 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 5/50
54/54 [==============================] - 175s 3s/step - loss: 0.7374 - accuracy: 0.5457 - precisi